# Conversando com os dados

In [2]:
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')


from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
caminhos = [
    "arquivos/Explorando o Universo das IAs com Hugging Face.pdf",
    "arquivos/Explorando a API da OpenAI.pdf",
    ]

paginas = []
for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())

recur_split = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(paginas)

for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('arquivos/', '')
    doc.metadata['doc_id'] = i

In [4]:
diretorio = 'arquivos/chat_retrieval_db'

embeddings_model = OpenAIEmbeddings()
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=diretorio
)

## Criando estrutura de conversa

In [5]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI(model='gpt-3.5-turbo-0125')

In [6]:
from langchain.chains.retrieval_qa.base import RetrievalQA

chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
)

In [7]:
pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
chat_chain.invoke({'query': pergunta})

{'query': 'O que é Hugging Face e como faço para acessá-lo?',
 'result': 'Hugging Face é uma plataforma que oferece uma variedade de modelos de inteligência artificial e ferramentas para trabalhar com eles. Para acessá-lo, você pode criar uma conta no site do Hugging Face e explorar os modelos disponíveis, solicitando acesso a eles conforme necessário. Cada modelo pode ter um processo específico para solicitar acesso, como concordar com termos de uso, por exemplo. Além disso, você pode utilizar as bibliotecas do Hugging Face, como transformers, para interagir com os modelos de forma mais prática.'}

## Modificando o prompt da chain

In [8]:
from langchain.prompts import PromptTemplate

chain_prompt = PromptTemplate.from_template(
"""Utilize o contexto fornceido para responder a pergunta ao final. 
Se você não sabe a resposta, apenas diga que não sabe e não tente inventar a resposta.
Utilize três frases no máximo, mantenha a resposta concisa.

Contexto: {context}

Pergunta: {question}

Resposta:
"""
)

In [9]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
    chain_type_kwargs={'prompt': chain_prompt},
    return_source_documents=True
)

In [11]:
pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])

Hugging Face é uma plataforma de IA com modelos e datasets disponíveis.
Para acessá-lo, é necessário criar uma conta e solicitar acesso aos modelos desejados.
O processo de acesso pode variar dependendo do modelo, mas geralmente envolve concordar com termos de uso.


In [13]:
resposta['source_documents']

[Document(page_content='Explorando o Universo das IAs com Hugging Face\nConteúdo\n01. O que é Hugging Face? 5\nAfinal, o que é Hugging Face? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\nÉ tudo aberto mesmo? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6\nComo usaremos o Hugging Face? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6\n02. A plataforma Hugging Face 7\nModelos . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 7\nPágina de um modelo . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 8\nDatasets . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 10\nSpaces . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 10\n03. Testando nossa primeira IA em tempo recorde 14\nNossa primeira IA . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 14', m

In [14]:
from langchain.globals import set_debug

set_debug(True)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})

set_debug(False)


[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "O que é Hugging Face e como faço para acessá-lo?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "O que é Hugging Face e como faço para acessá-lo?",
  "context": "Explorando o Universo das IAs com Hugging Face\nConteúdo\n01. O que é Hugging Face? 5\nAfinal, o que é Hugging Face? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\nÉ tudo aberto mesmo? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6\nComo usaremos o Hugging Face? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6\n02. A plataforma Hugging Face 7\nModelos . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 7\nPágina de um modelo . . . . . . . . . . . . . . 

## Outros tipos de chains

### Stuff

In [15]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
    chain_type='stuff'
)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])

Hugging Face é uma plataforma que disponibiliza uma variedade de modelos e datasets de Inteligência Artificial para uso. Para acessá-lo, você pode criar uma conta no site da plataforma e, em seguida, buscar por modelos específicos de interesse. Alguns modelos podem requerer solicitação de acesso, como no caso de modelos restritos, onde é necessário concordar com os termos de uso para utilizá-los. Além disso, é possível explorar a documentação e a comunidade do Hugging Face para obter mais informações e suporte sobre como utilizar a plataforma de forma eficiente.


### Map Reduce

In [16]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
    chain_type='map_reduce'
)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])

Hugging Face é uma plataforma de inteligência artificial que oferece uma variedade de modelos de IA pré-treinados para tarefas como processamento de linguagem natural, visão computacional e muito mais. Para acessá-lo, é necessário visitar o site oficial em https://huggingface.co/ e criar uma conta na plataforma. Lá você encontrará modelos pré-treinados, poderá treinar seus próprios modelos, explorar datasets e utilizar diversas ferramentas relacionadas à IA.


### Refine

In [17]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
    chain_type='refine'
)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])

Hugging Face é uma plataforma que oferece uma variedade de modelos de inteligência artificial pré-treinados, datasets e espaços de colaboração para a comunidade de IA. As bibliotecas do Hugging Face são extremamente práticas, permitindo aos usuários acessar e utilizar modelos de forma eficiente. Para acessá-lo, basta visitar o site oficial da Hugging Face em https://huggingface.co/. Lá você encontrará os modelos, datasets e demais recursos disponíveis para explorar e utilizar em seus projetos de IA. Além disso, você pode buscar na documentação ou comunidade do Hugging Face por funções ou ferramentas prontas que podem te ajudar a automatizar tarefas e facilitar seu trabalho com IA. Com o Hugging Face, você tem a oportunidade de combinar funcionalidades de diversos modelos para suas aplicações e integrar diferentes modelos em seus projetos de forma eficiente. As possibilidades são vastas e o céu é o limite quando se trata de explorar e utilizar os recursos oferecidos pelo Hugging Face.
